# me 3.Z Homework 001

##### Source  = https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/03-classification/homework.md

In [1]:
import pandas as pda

import numpy as npy

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
urlCsv = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv'
dfrHouRaw = pda.read_csv(urlCsv)
print(' shape = ' , dfrHouRaw.shape)
dfrHouRaw.head().T

 shape =  (20640, 10)


,0,1,2,3,4
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
latitude,37.88,37.86,37.85,37.85,37.85
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [3]:
# from homework:
lstCol = [ 
'latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value' ,
'ocean_proximity'
]
lstCol

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity']

### Prepare dfr Hou (Select fields + nan)

In [4]:
dfrHou = dfrHouRaw[lstCol]
dfrHou.head(8).T

,0,1,2,3,4,5,6,7
latitude,37.88,37.86,37.85,37.85,37.85,37.85,37.84,37.84
longitude,-122.23,-122.22,-122.24,-122.25,-122.25,-122.25,-122.25,-122.25
housing_median_age,41.0,21.0,52.0,52.0,52.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0,919.0,2535.0,3104.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0,213.0,489.0,687.0
population,322.0,2401.0,496.0,558.0,565.0,413.0,1094.0,1157.0
households,126.0,1138.0,177.0,219.0,259.0,193.0,514.0,647.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462,4.0368,3.6591,3.12
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0,269700.0,299200.0,241400.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [5]:
# do we have nulls ?
dfrHou.isnull().sum() [ dfrHou.isnull().sum() >= 1 ].sort_values(ascending=False)

total_bedrooms    207
dtype: int64

In [6]:
# Replace nan by 0
dfrHou.fillna(0, inplace=True)
dfrHou.isnull().sum() [ dfrHou.isnull().sum() >= 1 ].sort_values(ascending=False)

Series([], dtype: int64)

### Add fields

In [7]:
dfrHou['rooms_per_household'] = dfrHou.total_rooms / dfrHou.households
dfrHou['bedrooms_per_room'] = dfrHou.total_bedrooms / dfrHou.total_rooms
dfrHou['population_per_household'] = dfrHou.population / dfrHou.households

dfrHou.head(8).T

,0,1,2,3,4,5,6,7
latitude,37.88,37.86,37.85,37.85,37.85,37.85,37.84,37.84
longitude,-122.23,-122.22,-122.24,-122.25,-122.25,-122.25,-122.25,-122.25
housing_median_age,41.0,21.0,52.0,52.0,52.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0,919.0,2535.0,3104.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0,213.0,489.0,687.0
population,322.0,2401.0,496.0,558.0,565.0,413.0,1094.0,1157.0
households,126.0,1138.0,177.0,219.0,259.0,193.0,514.0,647.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462,4.0368,3.6591,3.12
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0,269700.0,299200.0,241400.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [8]:
dfrHou.columns

Index(['latitude', 'longitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household'],
      dtype='object')

## Q1 = Most frequent value of ocean_proximity

In [9]:
dfrHou.ocean_proximity.value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [10]:
dfrHou.ocean_proximity.mode()

0    <1H OCEAN
Name: ocean_proximity, dtype: object

## Q1 = '<1H OCEAN'

### Split data

In [11]:
dfrTraFul, dfrTst = train_test_split(dfrHou, test_size=0.2, random_state=42)
dfrTra, dfrVal = train_test_split(dfrTraFul, test_size=0.25, random_state=42)
dfrHou.shape , dfrTraFul.shape , dfrTra.shape, dfrVal.shape, dfrTst.shape

((20640, 13), (16512, 13), (12384, 13), (4128, 13), (4128, 13))

In [12]:
# Reset all index
dfrTraFul = dfrTraFul.reset_index(drop=True)  
dfrTra = dfrTra.reset_index(drop=True)
dfrVal = dfrVal.reset_index(drop=True)
dfrTst = dfrTst.reset_index(drop=True)

## Make median_house_value binary ( on TRA!! )
#### before removing predic variable - create a binary based on it

In [13]:
# for Q2 and ... store the Mean of median_house_value (before we drop it)
floHouMea = dfrHou.median_house_value.mean() 
floTraMea = dfrTra.median_house_value.mean() 
floHouMea , floTraMea , (floHouMea - floTraMea)

(206855.81690891474, 206807.7419250646, 48.0749838501506)

In [14]:
dfrTra.median_house_value

0        241400.0
1        500001.0
2         64100.0
3        277200.0
4        417000.0
           ...   
12379    154200.0
12380    146400.0
12381    215300.0
12382    139000.0
12383    181300.0
Name: median_house_value, Length: 12384, dtype: float64

In [15]:
# bigger than mean ?
(dfrTra.median_house_value > floHouMea).astype(int)

0        1
1        1
2        0
3        1
4        1
        ..
12379    0
12380    0
12381    1
12382    0
12383    0
Name: median_house_value, Length: 12384, dtype: int32

In [16]:
# see the values around the mean to double check
lstChk = (dfrTra.median_house_value > floHouMea - 100) & (dfrTra.median_house_value < floHouMea + 100) 
print('floHouVal = ' , floHouMea)
dfrTra.median_house_value[lstChk]

floHouVal =  206855.81690891474


5195     206900.0
5565     206900.0
5590     206800.0
6233     206800.0
7438     206900.0
8315     206900.0
11540    206900.0
Name: median_house_value, dtype: float64

In [17]:
# see our boolean for same recs
(dfrTra.median_house_value > floHouMea).astype(int)[lstChk]

5195     1
5565     1
5590     0
6233     0
7438     1
8315     1
11540    1
Name: median_house_value, dtype: int32

In [18]:
# add the column to dfr
dfrTra['above_average'] = (dfrTra.median_house_value > floHouMea).astype(int)
dfrTra['above_average'].describe()

count    12384.000000
mean         0.407623
std          0.491412
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: above_average, dtype: float64

In [19]:
dfrTra[ ['median_house_value' , 'above_average'] ]

,median_house_value,above_average
0,241400.0,1
1,500001.0,1
2,64100.0,0
3,277200.0,1
4,417000.0,1
...,...,...
12379,154200.0,0
12380,146400.0,0
12381,215300.0,1
12382,139000.0,0


### Add binary to VAL & TST too

In [20]:
# add the column to dfr VAL
dfrVal['above_average'] = (dfrVal.median_house_value > floHouMea).astype(int)
dfrVal['above_average'].describe()

count    4128.000000
mean        0.408188
std         0.491558
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: above_average, dtype: float64

In [21]:
# add the column to dfr TST
dfrTst['above_average'] = (dfrTst.median_house_value > floHouMea).astype(int)
dfrTst['above_average'].describe()

count    4128.000000
mean        0.400194
std         0.489997
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: above_average, dtype: float64

### Save Target variable (Binary !)

In [22]:
# save Target
# y_TraFul = dfrTraFul.above_average.values NOT calculated
y_Tra = dfrTra.above_average.values
y_Val = dfrVal.above_average.values
y_Tst = dfrTst.above_average.values

### Now we can (could? ;-) remove both Targets: median_house_value + above_average 

In [23]:
# Remove target n all dfr
#dfrTraFul.drop('median_house_value', axis=1 , inplace=True)
dfrTra.drop('median_house_value', axis=1 , inplace=True)
dfrVal.drop('median_house_value', axis=1 , inplace=True)
dfrTst.drop('median_house_value', axis=1 , inplace=True)

dfrTra.drop('above_average', axis=1 , inplace=True)
dfrVal.drop('above_average', axis=1 , inplace=True)
dfrTst.drop('above_average', axis=1 , inplace=True)


## Q2: Correlation

In [24]:
dfrTraCor = dfrTra.corr()
type(dfrTraCor)
print(dfrTraCor.shape)
dfrTraCor

(11, 11)


,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
latitude,1.000000,-0.925005,0.002477,-0.025914,-0.059730,-0.100272,-0.063529,-0.076805,0.119118,-0.124507,-0.002301
longitude,-0.925005,1.000000,-0.099812,0.036449,0.063840,0.091670,0.049762,-0.016426,-0.034814,0.102320,0.011022
housing_median_age,0.002477,-0.099812,1.000000,-0.363522,-0.324156,-0.292476,-0.306119,-0.119591,-0.181275,0.129456,0.012167
total_rooms,-0.025914,0.036449,-0.363522,1.000000,0.931546,0.853219,0.921441,0.198951,0.168926,-0.194185,-0.029452
total_bedrooms,-0.059730,0.063840,-0.324156,0.931546,1.000000,0.877340,0.979399,-0.009833,0.010381,0.078094,-0.034301
population,-0.100272,0.091670,-0.292476,0.853219,0.877340,1.000000,0.906841,-0.000849,-0.076210,0.031592,0.064998
households,-0.063529,0.049762,-0.306119,0.921441,0.979399,0.906841,1.000000,0.011925,-0.085832,0.058004,-0.032522
median_income,-0.076805,-0.016426,-0.119591,0.198951,-0.009833,-0.000849,0.011925,1.000000,0.394154,-0.616617,-0.000454
rooms_per_household,0.119118,-0.034814,-0.181275,0.168926,0.010381,-0.076210,-0.085832,0.394154,1.000000,-0.500589,0.001801
bedrooms_per_room,-0.124507,0.102320,0.129456,-0.194185,0.078094,0.031592,0.058004,-0.616617,-0.500589,1.000000,-0.002851


In [25]:
# Source= https://www.nbshare.io/notebook/394171759/How-To-Use-Pandas-Correlation-Matrix/
dfrTraCor[dfrTraCor < 1].unstack().transpose().sort_values( ascending=False).drop_duplicates()

total_bedrooms            households                  0.979399
total_rooms               total_bedrooms              0.931546
                          households                  0.921441
households                population                  0.906841
population                total_bedrooms              0.877340
                          total_rooms                 0.853219
rooms_per_household       median_income               0.394154
total_rooms               median_income               0.198951
                          rooms_per_household         0.168926
housing_median_age        bedrooms_per_room           0.129456
rooms_per_household       latitude                    0.119118
longitude                 bedrooms_per_room           0.102320
population                longitude                   0.091670
bedrooms_per_room         total_bedrooms              0.078094
population                population_per_household    0.064998
total_bedrooms            longitude                   0

In [26]:
# convert to unstack serie
serTraCorUst = dfrTraCor.unstack()
# drop the diagonbales = value = 1 (keep NOT + 1)
serTraCorUst = serTraCorUst[~(serTraCorUst==1)]
serTraCorUst

latitude                  longitude             -0.925005
                          housing_median_age     0.002477
                          total_rooms           -0.025914
                          total_bedrooms        -0.059730
                          population            -0.100272
                                                   ...   
population_per_household  population             0.064998
                          households            -0.032522
                          median_income         -0.000454
                          rooms_per_household    0.001801
                          bedrooms_per_room     -0.002851
Length: 110, dtype: float64

In [27]:
# Correlations in ABS value sorted desc - Top
#serTraCorUst.abs().sort_values( ascending=False).drop_duplicates()
serTraCorUst.abs().sort_values( ascending=False).drop_duplicates().head()

households      total_bedrooms    0.979399
total_bedrooms  total_rooms       0.931546
latitude        longitude         0.925005
households      total_rooms       0.921441
                population        0.906841
dtype: float64

### Q2 = households      total_bedrooms    0.979399

# Q3 Mutual information

In [28]:
dfrTra

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,34.43,-119.67,39.0,1467.0,381.0,1404.0,374.0,2.3681,<1H OCEAN,3.922460,0.259714,3.754011
1,33.74,-118.32,24.0,6097.0,794.0,2248.0,806.0,10.1357,NEAR OCEAN,7.564516,0.130228,2.789082
2,39.13,-121.62,41.0,1317.0,309.0,856.0,337.0,1.6719,INLAND,3.908012,0.234624,2.540059
3,34.24,-118.63,9.0,4759.0,924.0,1884.0,915.0,4.8333,<1H OCEAN,5.201093,0.194158,2.059016
4,37.52,-122.30,38.0,2769.0,387.0,994.0,395.0,5.5902,NEAR OCEAN,7.010127,0.139762,2.516456
...,...,...,...,...,...,...,...,...,...,...,...,...
12379,33.79,-118.29,16.0,1867.0,571.0,951.0,498.0,3.3427,<1H OCEAN,3.748996,0.305838,1.909639
12380,38.04,-121.34,16.0,3295.0,565.0,2279.0,576.0,3.6083,INLAND,5.720486,0.171472,3.956597
12381,32.74,-116.99,18.0,3341.0,611.0,1952.0,602.0,3.9844,<1H OCEAN,5.549834,0.182879,3.242525
12382,33.84,-117.87,16.0,1545.0,354.0,730.0,350.0,4.5112,<1H OCEAN,4.414286,0.229126,2.085714


In [29]:
# Our Categorical Feature
dfrTra.ocean_proximity.value_counts()

<1H OCEAN     5472
INLAND        3955
NEAR OCEAN    1558
NEAR BAY      1397
ISLAND           2
Name: ocean_proximity, dtype: int64

In [30]:
# M. importance of ocean_proximity on above_average (not any more in dfrTra)
mutual_info_score(dfrTra.ocean_proximity, y_Tra).round(2)

0.1

## Q3 = 0.10

# Q4: Logistic Reg

In [31]:
dfrTra.describe()

,latitude,longitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
count,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000,12384.000000
mean,35.642003,-119.579831,28.585837,2633.748950,537.035368,1427.289648,499.044089,3.870090,5.411837,0.213091,3.033264
std,2.130977,2.005077,12.608072,2137.057303,412.252061,1140.978664,375.749834,1.888237,2.077484,0.058301,7.074410
min,32.560000,-124.350000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,0.888889,0.100000,0.692308
25%,33.930000,-121.800000,18.000000,1461.000000,298.000000,790.000000,282.000000,2.562500,4.443167,0.175329,2.427379
50%,34.260000,-118.500000,29.000000,2127.000000,438.000000,1170.000000,412.000000,3.552100,5.223123,0.203165,2.816215
75%,37.720000,-118.010000,37.000000,3150.000000,647.250000,1726.000000,606.000000,4.740400,6.046054,0.239753,3.280020
max,41.950000,-114.310000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,61.812500,1.000000,599.714286


In [32]:
# Our Categorical Feature
dfrTra.ocean_proximity.value_counts()

<1H OCEAN     5472
INLAND        3955
NEAR OCEAN    1558
NEAR BAY      1397
ISLAND           2
Name: ocean_proximity, dtype: int64

In [33]:
# create an instance of dictvectorizer
#                    no sparse matrixes
dvt = DictVectorizer(sparse=False)

In [34]:
# create a dictionary with all fileds and values - one entry by record
dicTra = dfrTra.to_dict(orient='records')
dicTra[25:27]

[{'latitude': 34.19,
  'longitude': -118.3,
  'housing_median_age': 14.0,
  'total_rooms': 3615.0,
  'total_bedrooms': 913.0,
  'population': 1924.0,
  'households': 852.0,
  'median_income': 3.5083,
  'ocean_proximity': '<1H OCEAN',
  'rooms_per_household': 4.242957746478873,
  'bedrooms_per_room': 0.25255878284923927,
  'population_per_household': 2.2582159624413145},
 {'latitude': 37.58,
  'longitude': -122.0,
  'housing_median_age': 6.0,
  'total_rooms': 4405.0,
  'total_bedrooms': 717.0,
  'population': 2071.0,
  'households': 688.0,
  'median_income': 5.8151,
  'ocean_proximity': '<1H OCEAN',
  'rooms_per_household': 6.402616279069767,
  'bedrooms_per_room': 0.16276958002270148,
  'population_per_household': 3.010174418604651}]

In [35]:
X_Tra = dvt.fit_transform(dicTra)
X_Tra.shape

(12384, 16)

In [36]:
# to Check the names of the column
dvt.get_feature_names_out()

array(['bedrooms_per_room', 'households', 'housing_median_age',
       'latitude', 'longitude', 'median_income',
       'ocean_proximity=<1H OCEAN', 'ocean_proximity=INLAND',
       'ocean_proximity=ISLAND', 'ocean_proximity=NEAR BAY',
       'ocean_proximity=NEAR OCEAN', 'population',
       'population_per_household', 'rooms_per_household',
       'total_bedrooms', 'total_rooms'], dtype=object)

In [37]:
# same for VAL dataset
dicVal = dfrVal.to_dict(orient='records')
X_Val = dvt.transform(dicVal)
X_Val.shape

(4128, 16)

## Log Reg

In [38]:
# Double chek LOG Reg Target is binary {0,1} id not LofRev will run forever !!!!
y_Tra.min() , y_Tra.max()

(0, 1)

In [39]:
X_Tra.shape , y_Tra.shape

((12384, 16), (12384,))

In [40]:
modLoR = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
modLoR.fit(X_Tra, y_Tra)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [41]:
# what is w0 (return npy array)
w0 = modLoR.intercept_[0]
w0

-0.06226206219069978

In [42]:
wi = modLoR.coef_[0]
wi

array([ 1.13415479e-01,  4.04233123e-03,  3.57188030e-02,  1.02626550e-01,
        8.27375706e-02,  1.19638568e+00,  4.52128771e-01, -1.59850879e+00,
        1.19112064e-02,  3.09190545e-01,  7.63016201e-01, -1.63226052e-03,
        1.04764314e-02, -1.36661587e-02,  1.80979534e-03, -1.53342014e-04])

### Predictions on VAL

In [43]:
# Let's predict above_average (0,1) for X_Val data (return npy array)
modLoR.predict(X_Val)

array([0, 0, 1, ..., 1, 1, 0])

In [44]:
# number of house that would be above mean (prediction) in VAL:
modLoR.predict(X_Val).sum() , modLoR.predict(X_Val).shape[0]

(1668, 4128)

In [45]:
# What is real in VAL
y_Val.sum() 

1685

In [46]:
# Let's predict VAL with probability
y_pred = modLoR.predict_proba(X_Val)
y_pred.round(3)

array([[0.912, 0.088],
       [0.84 , 0.16 ],
       [0.048, 0.952],
       ...,
       [0.041, 0.959],
       [0.151, 0.849],
       [0.528, 0.472]])

In [47]:
# Predict but taking only the Probability of 1
y_predOne = modLoR.predict_proba(X_Val)[:,1]
y_predOne.round(3)

array([0.088, 0.16 , 0.952, ..., 0.959, 0.849, 0.472])

In [48]:
# detect when Pred of 1 >= 0.5
price_above = (y_predOne >= 0.5)
price_above

array([False, False,  True, ...,  True,  True, False])

In [49]:
price_above.sum()

1668

In [50]:
# store this Accuracy for Q5 (elimination technique)
floAcc = (y_Val == price_above).mean()
floAcc

0.8359980620155039

In [51]:
(y_Val == price_above).mean().round(2)

0.84

## Q4 = 0.84

# Q5: Feature elimination

In [52]:
lstCol = list(dfrTra.columns.values)
lstCol

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'ocean_proximity',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [53]:
# Dataframe to save result index=feature name
dfrRes = pda.DataFrame(columns=[ 'Glo_Accuracy' , 'Tmp_Accuracy',  'Delta' , 'Abs_Delta'])

# one extra iteration to re-execute same LoR without droppinng any feature (double check)
cstGlo = '*Glo*'

for col in lstCol + [cstGlo]:
    if col != cstGlo:
        print('--- Dropping' , col )
    else:
        print('--- No Dropping for check ' , cstGlo  )
    
    # copy the dfr in a Tmp version
    dfrTraTmp = dfrTra.copy()
    
    # remove this column (except last)
    if col != cstGlo:
        dfrTraTmp.drop(col, axis=1, inplace=True)

    #print(list(dfrTraTmp.columns.values))
        
    # create dictionay
    dicTraTmp = dfrTraTmp.to_dict(orient='records')
    #print(dicTraTmp[25:26])   # au hasard ;-)
    
    # OHE
    X_TraTmp = dvt.fit_transform(dicTraTmp)
    #print(X_TraTmp[25:26])
    
    # LogReg
    modLoR = LogisticRegression(solver="liblinear", C=1.0, max_iter=1000, random_state=42)
    modLoR.fit(X_TraTmp, y_Tra)
    
    # same for VAL dataset
    dfrValTmp = dfrVal.copy()
    if col != cstGlo:
        dfrValTmp.drop(col, axis=1, inplace=True)
    dicValTmp = dfrValTmp.to_dict(orient='records')
    X_ValTmp = dvt.transform(dicValTmp)
    #print(X_ValTmp.shape)
    
    # Predict but taking only the Probability of 1
    y_predOneTmp = modLoR.predict_proba(X_ValTmp)[:,1]
    #print(y_predOneTmp.round(3))
    
    # detect when Pred of 1 >= 0.5
    price_above_tmp = (y_predOneTmp >= 0.5)
    #print(price_above_tmp)
    
    # calculate accuracy
    floAccTmp = (y_Val == price_above_tmp).mean()
    
    # add the difference to full Val Accuracy 
    print('      floAcc= ', floAcc , ' floAccTmp=' , floAccTmp , ' diff=' , (floAccTmp - floAcc) )
    #                             ['Feature', 'Glo_Accuracy' , 'Tmp_Accuracy',  'Delta' , 'Abs_Delta']
    dfrRes.loc[col] = [ floAcc  , floAccTmp  , floAcc-floAccTmp ,  abs(floAcc-floAccTmp) ] 

--- Dropping latitude
      floAcc=  0.8359980620155039  floAccTmp= 0.8323643410852714  diff= -0.003633720930232509
--- Dropping longitude
      floAcc=  0.8359980620155039  floAccTmp= 0.8318798449612403  diff= -0.004118217054263518
--- Dropping housing_median_age
      floAcc=  0.8359980620155039  floAccTmp= 0.8316375968992248  diff= -0.0043604651162790775
--- Dropping total_rooms
      floAcc=  0.8359980620155039  floAccTmp= 0.8362403100775194  diff= 0.00024224806201555982
--- Dropping total_bedrooms
      floAcc=  0.8359980620155039  floAccTmp= 0.8372093023255814  diff= 0.001211240310077577
--- Dropping population
      floAcc=  0.8359980620155039  floAccTmp= 0.8263081395348837  diff= -0.009689922480620172
--- Dropping households
      floAcc=  0.8359980620155039  floAccTmp= 0.8340600775193798  diff= -0.0019379844961240345
--- Dropping median_income
      floAcc=  0.8359980620155039  floAccTmp= 0.7853682170542635  diff= -0.050629844961240345
--- Dropping ocean_proximity
      floAcc

In [54]:
dfrRes.sort_values(by='Delta')

,Glo_Accuracy,Tmp_Accuracy,Delta,Abs_Delta
total_bedrooms,0.835998,0.837209,-0.001211,0.001211
total_rooms,0.835998,0.836240,-0.000242,0.000242
bedrooms_per_room,0.835998,0.836240,-0.000242,0.000242
*Glo*,0.835998,0.835998,0.000000,0.000000
population_per_household,0.835998,0.835756,0.000242,0.000242
rooms_per_household,0.835998,0.835271,0.000727,0.000727
households,0.835998,0.834060,0.001938,0.001938
latitude,0.835998,0.832364,0.003634,0.003634
longitude,0.835998,0.831880,0.004118,0.004118
housing_median_age,0.835998,0.831638,0.004360,0.004360


### Q5 Smallest in the proposed list = 

In [55]:
lstQ5 = [ 
    'total_rooms' ,
    'total_bedrooms' ,
    'population' ,
    'households'
    ]
lstQ5

['total_rooms', 'total_bedrooms', 'population', 'households']

In [56]:
dfrRes[ dfrRes.index.isin(lstQ5) ].sort_values(by='Abs_Delta')

,Glo_Accuracy,Tmp_Accuracy,Delta,Abs_Delta
total_rooms,0.835998,0.836240,-0.000242,0.000242
total_bedrooms,0.835998,0.837209,-0.001211,0.001211
households,0.835998,0.834060,0.001938,0.001938
population,0.835998,0.826308,0.009690,0.009690


## Q5: 'total_rooms'  (in ABS !! )
## Note: 'total_bedrooms' is smallest value (<0)

# Q6: SciKit

In [57]:
# Check columns in global dataset
dfrHou.columns , dfrHou.shape

(Index(['latitude', 'longitude', 'housing_median_age', 'total_rooms',
        'total_bedrooms', 'population', 'households', 'median_income',
        'median_house_value', 'ocean_proximity', 'rooms_per_household',
        'bedrooms_per_room', 'population_per_household'],
       dtype='object'),
 (20640, 13))

In [58]:
dfrHou.median_house_value.describe()

count     20640.000000
mean     206855.816909
std      115395.615874
min       14999.000000
25%      119600.000000
50%      179700.000000
75%      264725.000000
max      500001.000000
Name: median_house_value, dtype: float64

In [59]:
dfrHou['median_house_value_l1p'] = npy.log1p(dfrHou['median_house_value'])
dfrHou.median_house_value_l1p.describe()

count    20640.000000
mean        12.084891
std          0.569131
min          9.615805
25%         11.691916
50%         12.099050
75%         12.486451
max         13.122367
Name: median_house_value_l1p, dtype: float64

In [60]:
# Split data
dfrTraFul, dfrTst = train_test_split(dfrHou, test_size=0.2, random_state=42)
dfrTra, dfrVal = train_test_split(dfrTraFul, test_size=0.25, random_state=42)
dfrHou.shape , dfrTraFul.shape , dfrTra.shape, dfrVal.shape, dfrTst.shape

((20640, 14), (16512, 14), (12384, 14), (4128, 14), (4128, 14))

In [61]:
# Reset all index
dfrTraFul = dfrTraFul.reset_index(drop=True)  
dfrTra = dfrTra.reset_index(drop=True)
dfrVal = dfrVal.reset_index(drop=True)
dfrTst = dfrTst.reset_index(drop=True)

In [62]:
# take our target with l1p !!
y_TraFul = dfrTraFul.median_house_value_l1p
y_Tra = dfrTra.median_house_value_l1p
y_Val = dfrVal.median_house_value_l1p
y_Tst = dfrTst.median_house_value_l1p

In [63]:
dfrTra.columns

Index(['latitude', 'longitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household',
       'median_house_value_l1p'],
      dtype='object')

In [64]:
dfrTraFul.drop(['median_house_value' , 'median_house_value_l1p' ] , axis='columns' , inplace=True)
dfrTra.drop(['median_house_value' , 'median_house_value_l1p' ] , axis='columns' , inplace=True)
dfrVal.drop(['median_house_value' , 'median_house_value_l1p' ] , axis='columns' , inplace=True)
dfrTst.drop(['median_house_value' , 'median_house_value_l1p' ] , axis='columns' , inplace=True)

In [65]:
dicTra = dfrTra.to_dict(orient='records')
dicTra[24:25]

[{'latitude': 37.99,
  'longitude': -122.33,
  'housing_median_age': 4.0,
  'total_rooms': 3999.0,
  'total_bedrooms': 1079.0,
  'population': 1591.0,
  'households': 887.0,
  'median_income': 3.911,
  'ocean_proximity': 'NEAR BAY',
  'rooms_per_household': 4.508455467869222,
  'bedrooms_per_room': 0.2698174543635909,
  'population_per_household': 1.7936865839909808}]

In [66]:
dvt = DictVectorizer(sparse=False)
X_Tra = dvt.fit_transform(dicTra)
X_Tra.shape

(12384, 16)

In [67]:
# to Check the names of the column
dvt.get_feature_names_out()

array(['bedrooms_per_room', 'households', 'housing_median_age',
       'latitude', 'longitude', 'median_income',
       'ocean_proximity=<1H OCEAN', 'ocean_proximity=INLAND',
       'ocean_proximity=ISLAND', 'ocean_proximity=NEAR BAY',
       'ocean_proximity=NEAR OCEAN', 'population',
       'population_per_household', 'rooms_per_household',
       'total_bedrooms', 'total_rooms'], dtype=object)

In [68]:
# same for VAL dataset
dicVal = dfrVal.to_dict(orient='records')
X_Val = dvt.transform(dicVal)
X_Val.shape

(4128, 16)

In [69]:
# from Homework:
lstAlp = [0, 0.01, 0.1, 1, 10]
#lstAlp = [0]

In [70]:
# Dataframe to save result index=feature name
dfrRes = pda.DataFrame(columns=[ 'RMSE' , 'RMSE_R3'])

# loop each value of alpha
for alp in lstAlp:
    print('--- Using alpha = ' , alp)
    
    # LiR
    modLiR = Ridge(alpha=alp, solver="sag", random_state=42)
    modLiR.fit(X_Tra, y_Tra)
    
    # predict VAL
    y_Prd = modLiR.predict(X_Val)
    
    # Calc RMSE with sickit
    floMse = mean_squared_error(y_Val, y_Prd) 
    
    print('    floMse = ' , floMse )
    
    # Store Result in dfrRes
    dfrRes.loc[alp] = [ floMse , floMse.round(3) ] 

print('*** Done ***')

--- Using alpha =  0
    floMse =  0.27464262613642076
--- Using alpha =  0.01
    floMse =  0.27464262615435947
--- Using alpha =  0.1
    floMse =  0.274642626324762
--- Using alpha =  1
    floMse =  0.27464262803776485
--- Using alpha =  10
    floMse =  0.27464264514086784
*** Done ***


In [71]:
# force display to 10 digits to verify that it changes ;-)
pda.options.display.precision = 10
dfrRes

,RMSE,RMSE_R3
0.00,0.2746426261,0.275
0.01,0.2746426262,0.275
0.10,0.2746426263,0.275
1.00,0.2746426280,0.275
10.00,0.2746426451,0.275


#### "If there are multiple options, select the smallest alpha."
## Q6 = 0   (all at 0.275)